In [3]:
import newspaper
from newspaper import Article
from newspaper import fulltext
import requests
import pandas as pd
import lxml.etree
import lxml.html
import lxml.html.clean
import datetime
import re
from lxml import html
from urllib.parse import urlparse
import codecs
from io import StringIO
from io import BytesIO

In [9]:
df = pd.read_csv(r"C:\Users\dell\Downloads\urls - content_details.csv")
urls = df['url']
urls = urls.dropna().to_list()

In [33]:
def get_article_urls(main_url):
    try:
        article_urls = []
        main_page = newspaper.build(main_url)
        for article in main_page.articles:
            article_urls.append(article.url)
        
        return article_urls if article_urls else None
        
    except Exception as e:
        print("Error: ", e)

def remove_citations(text):   
    citation_patterns = [
        r'\[\d+\]',     
        r'\(\w+\s*,\s*\d+\)' 
    ]  
    combined_pattern = '|'.join(citation_patterns)
    cleaned_text = re.sub(combined_pattern, '', text)

    return cleaned_text

def remove_n(text):
    return text.replace("\n", ' ')

def remove_t(text):
    return text.replace('\t', ' ')  

def extract_tables(html_content):
    tree = html.fromstring(html_content)
    tables = tree.xpath('//table')
    table_data = []
    if tables:
        for table in tables:                      
            df = pd.read_html(html.tostring(table), header=0, encoding='utf-8')[0]
            df.fillna('-', inplace=True)    
                
            table_data.append(df.to_dict(orient='records'))

        return table_data
    else:
        return None
    
def replace_nonbreaking_spaces(text):
    return re.sub(r'\s+', ' ', text)

def extract_domain(url):
    parsed_url = urlparse(url)
    return parsed_url.netloc.split('.')[1]

def encoding_text(text):
    encoded_text = text.encode('utf-8')
    decoded_text = encoded_text.decode('utf-8')     
    return decoded_text

def clean_special_char(text):
    return re.sub(r'\s+', ' ', text)

In [28]:
txt = """experience\u2014such as making check-in and boarding processes more 
seamless\u2014and support services\u2014from revenue accounting to 
invoicing\u2014to drive the next level  carrier\u2019s cost 
Do you want to learn more about hotel marketing? In the articles \u201cThe 
Latest Hotel Marketing Trends in the Hotel Industry\u201d and \u201cEssential 
Hotel Marketing Strategies,\u201d you find the latest hotel marketing trends and 
strategies. Hotel Revenue Management Hotel revenue management enables owners to 
predict demand levels and optimize distribution and pricing to maximize financial 
results. In the article \u201cWhat is Hotel Revenue Management?\u201d, you will 
find out more about hotel revenue management, the reasons it is so valuable to 
hotel owners, and the necessary conditions for implementing a hotel revenue 
management strategy. Hotel Technology Trends Hotel technology has advanced in 
leaps and bounds in recent years. Not only are \u201csmart\u201d systems entirely 
prevalent, but guests can expect to enjoy more interactive experiences than ever 
before. In the article \u201cThe Latest Hotel Technology Trends,\u201d you """
# txt = replace_nonbreaking_spaces(txt)
print(txt)

experience—such as making check-in and boarding processes more 
seamless—and support services—from revenue accounting to 
invoicing—to drive the next level  carrier’s cost 
Do you want to learn more about hotel marketing? In the articles “The 
Latest Hotel Marketing Trends in the Hotel Industry” and “Essential 
Hotel Marketing Strategies,” you find the latest hotel marketing trends and 
strategies. Hotel Revenue Management Hotel revenue management enables owners to 
predict demand levels and optimize distribution and pricing to maximize financial 
results. In the article “What is Hotel Revenue Management?”, you will 
find out more about hotel revenue management, the reasons it is so valuable to 
hotel owners, and the necessary conditions for implementing a hotel revenue 
management strategy. Hotel Technology Trends Hotel technology has advanced in 
leaps and bounds in recent years. Not only are “smart” systems entirely 
prevalent, but guests can expect to enjoy more interactive experie

In [34]:
url ="https://www.transportpolicy.net/standard/india-fuels-diesel-and-gasoline/"
article = Article(url)
article.download()
article.parse()
article.nlp()
tables = extract_tables(article.html)
tables

[[{'Date': 1995, 'Diesel': '10,000 ppm (nationwide)', 'Gasoline': '–'},
  {'Date': 1996,
   'Diesel': '5,000 ppm (Delhi + selected cities)',
   'Gasoline': '–'},
  {'Date': 1998, 'Diesel': '2,500 ppm (Delhi)', 'Gasoline': '–'},
  {'Date': 1999,
   'Diesel': '500 ppm (BS II, Delhi, limited supply)',
   'Gasoline': '–'},
  {'Date': 2000, 'Diesel': '2,500 ppm (nationwide)', 'Gasoline': '–'},
  {'Date': 2001,
   'Diesel': '500 ppm (BS II, selected cities)',
   'Gasoline': '–'},
  {'Date': 2005,
   'Diesel': '500 ppm (BS II, nationwide) 350 ppm (BS III, selected cities)',
   'Gasoline': '500 ppm (BS II, nationwide) 150 ppm (BS III, selected cities)'},
  {'Date': 2010,
   'Diesel': '350 ppm (BS III; nationwide) 50 ppm (BS IV; selected cities)',
   'Gasoline': '150 ppm (BS III, nationwide) 50 ppm (BS IV, selected cities)'},
  {'Date': 2017,
   'Diesel': '50 ppm (BS IV;nationwide)',
   'Gasoline': '50 ppm (BS IV; nationwide)'},
  {'Date': 2020,
   'Diesel': '10ppm (BS VI; nationwide)',
   'Gas

In [24]:
url = 'http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/'
# url = 'https://www.worldbank.org/en/topic/agriculture/brief/food-security-update'
article = Article(urls[0])
article.download()
article.parse()

In [35]:
def newspaper_article(url):
    try:  
        article = Article(url) 
        article.download()       
        article.parse()
        article.nlp()
        
        text = article.text        
        text = remove_citations(text)
        text = remove_n(text)
        text = remove_t(text)
        text = encoding_text(text)
        text = clean_special_char(text)

        title = article.title       
        title = remove_citations(title)
        title = remove_n(title)
        title = remove_t(title)
        title = encoding_text(title)
        title = clean_special_char(title)
        
        summary = article.summary       
        summary = remove_citations(summary)
        summary = remove_n(summary)
        summary = remove_t(summary)
        summary = encoding_text(summary)
        summary = clean_special_char(summary)
        
        result = {
        "Domain": extract_domain(url),
        "Published date": article.publish_date.strftime("%d/%m/%Y") if article.publish_date else None,
        "Authors": article.authors if article.authors else None,
        'Keywords': article.keywords if article.keywords else None,
        "Title": title if title else None,
        'Summary': summary if summary else None,
        "Content": {"heading": title if title else None,
                    "text": text if text else None,
                    }
                } 
        tables = extract_tables(article.html)
        if tables:
            result['Content']['tables'] = tables    
        return result
         
    except Exception as e:
        print("Error: ", e)

In [107]:
url = 'https://www.sportsunfold.com/top-10-best-cricket-tournament/'
res = newspaper_article(url)
res

{'Domain': 'sportsunfold',
 'Published date': '04/01/2023',
 'Authors': ['Raja Singh'],
 'Keywords': ['best',
  'india',
  'tournament',
  'right',
  'team',
  'edition',
  'international',
  'world',
  'successful',
  'cricket',
  'tournaments'],
 'Title': 'Top 10 Best Cricket Tournament in the World Right Now',
 'Summary': 'So now let’s check out the complete details about the Top 10 Best Cricket Tournament n the World Right Now. Top 10 Best Cricket Tournament in the WorldCheck Here the Top 10 Best Cricket Tournaments in the world right now. ICC Cricket World CupIt is one of the most prestigious biggest tournaments that take place every four years and also home to the best match in the world cup. Administration: BCCI, Cricket Australia, and Cricket South AfricaFormat: Twenty20First Edition: 2008Most Successful Team: Chennai Super Kings & Mumbai Indians5. Administration: Board of Control for Cricket in India (BCCI) & Cricket AustraliaFormat: Test CricketFirst Edition: 1996-97Most Succ

In [29]:
headings = tree.xpath('//h1 | //h2 | //h3 | //h4 | //h5')
result = {}
for heading in headings:
    # Find all elements following the current heading until the next heading
    following_elements = heading.xpath('following-sibling::*[preceding-sibling::h1 or preceding-sibling::h2 or preceding-sibling::h3 or preceding-sibling::h4 or preceding-sibling::h5 or preceding-sibling::h6][1]')

    # Extract text from each following element
    extracted_text = []
    for el in following_elements:
        text = el.text_content().strip().replace("\t", '').replace("\n",'').replace("  ",'')
        if text:
            extracted_text.append(text)

    # Store the result in the dictionary
    result[heading.text_content().replace("\t", '').replace("\n",'').replace("  ",'')] = extracted_text
result

{'Back to the future? Airline sector poised for change post-COVID-19': [],
 '': ['Jaap Bouwer is a senior expert in McKinsey’s Amsterdam office, Steve Saxon is a partner in the Shenzhen office, and Nina Wittkamp is a partner in the Munich office. The authors wish to thank Alex Dichter and Vik Krishnan for their contributions to this article.This article was edited by Jason Li, a senior editor in the Shanghai office.'],
 '1. Leisure trips will fuel the recovery': ['Business travel will take longer to recover, and even then, we estimate it will only likely recover to around 80 percent of prepandemic levels by 2024. Remote work\xa0and other flexible working arrangements are likely to remain in some form postpandemic and people will take fewer corporate trips.'],
 'Watch': ["A McKinsey Live event on 'Returning to corporate travel: How do we get it right?'"],
 'The call: Revisit flight economics': ['Airlines should reevaluate the economics of their operations, especially long-haul flights. 

In [10]:
from lxml import html
import requests

def extract_header_para(url):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        html_content = response.content.decode('utf-8', 'ignore')

        try:
            tree = html.fromstring(html_content)
        except Exception as e:
            print(f"Error parsing HTML: {e}")
            print("url" , url)
            return {f"Error parsing HTML: {e}"}

        headers = tree.xpath('//h1 | //h2 | //h3 | //h4 | //h5')

        result = {}
        for header in headers:
            paragraphs = header.xpath('following-sibling::p')
            result[header.text_content()] = [p.text_content() for p in paragraphs]
        if len(result) ==0:
            print("url with empty data: " , url)
            return {f"heading and para tags not found even with 200 status code"}
        
        else:
            return result
    else:
        print(f'html page could not fetched {response.status_code}')
        print("url" , url)
        return {f"html page could not fetched  {response.status_code}"}

#issues
# if there are two or more headers for same paragraph the paragraph is repeating for both the headers
# text celaning required for special characters, smileys and images e.g '\xa0'
# if paragraph is missing returns empty list
# read more scenario need to handle
# tables are not scraped



In [108]:
urls_= urls[:60]
len(urls_)

60

In [127]:
artcle_data = []
for url in urls_:
    try:
        result = newspaper_article(url)
        artcle_data.append({'url': url,
                            'data':result})
    except Exception as e:
        print(url, e)
        artcle_data.append({'url': url,
                            'data':e})

C:\Users\dell\AppData\Local\Temp\ipykernel_8444\1757940591.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-', inplace=True)
C:\Users\dell\AppData\Local\Temp\ipykernel_8444\1757940591.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-', inplace=True)


Error:  Article `download()` failed with 403 Client Error: Forbidden for url: https://www.pnas.org/doi/full/10.1073/pnas.2006991117 on URL https://www.pnas.org/doi/full/10.1073/pnas.2006991117
Error:  Article `download()` failed with 403 Client Error: Forbidden for url: https://ca.indeed.com/career-advice/finding-a-job/education-vs-experience on URL https://ca.indeed.com/career-advice/finding-a-job/education-vs-experience
Error:  Article `download()` failed with 403 Client Error: Forbidden for url: https://www.un.org/en/un75/new-era-conflict-and-violence on URL https://www.un.org/en/un75/new-era-conflict-and-violence
Error:  Missing optional dependency 'html5lib'.  Use pip or conda to install html5lib.
Error:  Article `download()` failed with 403 Client Error: Forbidden for url: https://www.indeed.com/career-advice/career-development/types-of-networks on URL https://www.indeed.com/career-advice/career-development/types-of-networks
Error:  Article `download()` failed with 403 Client Err

In [244]:
text = 'Russia\u2019s'
re.sub('\u2019', '"', text)

'Russia"s'

In [189]:
ur = 'https://www.pnas.org/doi/full/10.1073/pnas.2006991117'
art = Article(ur)
art.download()

In [122]:
import json
with open('newspaper.json', 'w') as file:
    json.dump(artcle_data, file)

In [37]:
data1 = []
for i, url in enumerate(urls_):
    res = extract_header_para(url)
    data1.append({"url": url,                  
                  "data": res})
    # print(i)
    

html page could not fetched 403
url https://www.indeed.com/career-advice/career-development/types-of-networks


In [102]:
ur ='https://www.ibm.com/topics/quantum-computing#:~:text=Quantum%20computing%20is%20a%20rapidly,hundreds%20of%20thousands%20of%20developers'
ur = 'https://science.howstuffworks.com/ten-nasa-achievements.htm'

In [103]:
res = extract_header_para(ur)
res

{"NASA's 10 Greatest Achievements": [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t10: Explorer 1, the First U.S. Satellite\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t9: Hubble Space Telescope, the Universe Unveiled\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t8: Chandra X-ray Observatory, the High-energy Universe\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t7: The Juno Spacecraft, a Trip to Jupiter\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t6: Apollo 13, Brilliance at Mission Control\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t5: The Space Shuttle, a Reusable Spacecraft\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t4: The Apollo 8 Mission, a Lunar Christmas Eve\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t3: The Mars Science Laboratory, a Rockhound on the Red Planet\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t2: Freedom 7, the First American in Space\n\n\t\n\t\t': [],
 '\n\t\t\t\n\n\n\n\n\t\t\t\n\t\n\t\t1: Apollo 11

In [39]:
def extract_all_data(url):    
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers)
    # print('response: ', response.status_code)
    html_content = response.content.decode('utf-8', 'ignore')
    tree = html.fromstring(html_content)
    headings = tree.xpath('//h1 | //h2 | //h3 |//h4 | //h5')
    # print('headers: ', headers)

    result = {}

    for heading in headings:
        # Find all elements following the current heading until the next heading
        following_elements = heading.xpath('following-sibling::*[preceding-sibling::h1 or preceding-sibling::h2 or preceding-sibling::h3 or preceding-sibling::h4 or preceding-sibling::h5 or preceding-sibling::h6][1]')

        # Extract text from each following element
        extracted_text = []
        for el in following_elements:
            text = el.text_content().strip().replace("\t", '').replace("\n",'').replace("  ",'')
            if text:
                extracted_text.append(text)

        # Store the result in the dictionary
        result[heading.text_content().replace("\t", '').replace("\n",'').replace("  ",'')] = extracted_text
    return result


In [10]:
false_url =[
'https://www.sankalpindia.net/history-indian-space-research',
'https://science.howstuffworks.com/ten-nasa-achievements.htm',
'https://www.geographynotes.com/forest/importance/economic-importance-of-forests-india-geography/8179',
'https://www.paris2024.org/en/history-of-olympic-games/',
'https://pib.gov.in/PressReleaseIframePage.aspx?PRID=1847066',
'https://kids.nationalgeographic.com/geography/countries/article/india#:~:text=continent%20of%20Asia.-,Most%20of%20India%20forms%20a%20peninsula%2C%20which%20means%20it%20is,bordered%20by%20the%20Arabian%20Sea.',
'https://walnutfolks.com/coke-vs-pepsi-evolution-of-branding/#:~:text=Coca%20Cola%20aka%20Coke%20was,in%20the%20marketing%20efforts%20began.',
'https://www.un.org/en/un75/new-era-conflict-and-violence',
'https://www.brahmakumaris.com/blog/jal-jan/importance-of-water-conservation/',
'https://risingkashmir.com/social-media-a-boon-or-a-bane',
'https://www.survivalinternational.org/articles/50facts',
'https://www.jagranjosh.com/general-knowledge/summary-of-ancient-indian-dynasties-and-their-contributions-1470736419-1',
'https://education.nationalgeographic.org/resource/key-components-civilization/',
'https://www.indeed.com/career-advice/career-development/types-of-networks',
'https://www.noaa.gov/education/resource-collections/marine-life',
'https://www.linkedin.com/pulse/indias-emergence-global-leader-digital-revolution-ganesh'
]

In [46]:
data1 = []
for i, url in enumerate(urls_):
    try:
        res = extract_data(url)
        data1.append({"url": url,                  
                    "data": res})
    except Exception as e:
        data1.append({"url": url,
                      "error": e})
        print(url, e)
 
   

https://www.pnas.org/doi/full/10.1073/pnas.2006991117 'NoneType' object has no attribute 'xpath'
https://www.investopedia.com/terms/t/third-world.asp 'NoneType' object has no attribute 'xpath'
https://ca.indeed.com/career-advice/finding-a-job/education-vs-experience 'NoneType' object has no attribute 'xpath'
https://www.un.org/en/un75/new-era-conflict-and-violence 'NoneType' object has no attribute 'xpath'
https://hidayatrizvi.com/corporate-life-vs-entrepreneurship/ 'NoneType' object has no attribute 'xpath'
https://www.indeed.com/career-advice/career-development/types-of-networks 'NoneType' object has no attribute 'xpath'


In [31]:
rs1 = []
for url in false_url:
    try:
        result = extract_data(url)
        rs1.append({
            'url': url,
            'data':result})
    except Exception as e:
        print(url, e)

https://www.un.org/en/un75/new-era-conflict-and-violence 'NoneType' object has no attribute 'xpath'
https://www.brahmakumaris.com/blog/jal-jan/importance-of-water-conservation/ 'NoneType' object has no attribute 'xpath'
https://www.noaa.gov/education/resource-collections/marine-life 'NoneType' object has no attribute 'xpath'


In [35]:
for r in rs1:
    print(r)

{'url': 'https://www.sankalpindia.net/history-indian-space-research', 'data': "India's experience in rocketry began in ancient times when fireworks were first used in the country, a technology invented in neighbouring China, and which had an extensive two-way exchange of ideas and goods with India, connected by the Silk Road. Military use of rockets by Tipu Sultan during the Mysore War against the British inspired William Congreve to invent the Congreve rocket, predecessor of modern artillery rockets, in 1804. After India gained independence from British occupation in 1947, Indian scientists and politicians recognized the potential of rocket technology in both defence applications, and for research and development. Recognizing that a country as demographically large as India would require its own independent space capabilities, and recognising the early potential of satellites in the fields of remote sensing and communication, these visionaries set about establishing a space research o

In [23]:
rs = []
for url in false_url:
    result = extract_all_data(url)
    rs.append({
        'url': url,
        'data':result})

In [26]:
u = 'https://www.paris2024.org/en/history-of-olympic-games/'
u = rs[8]['url']
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get(u, headers=headers)
# print('response: ', response.status_code)
html_content = response.content.decode('utf-8', 'ignore')
tree = html.fromstring(html_content)
text = tree.text_content().strip().replace("\n", '').replace("  ","")
print(text)

		 {{playListTitle}}{{ playLabelButton }}{{ index + 1 }}{{ track.track_title }}{{ track.track_artist }}{{ track.album_title }}{{ track.lenght }}{{list.tracks[currentTrack].track_title}}{{list.tracks[currentTrack].track_artist && typeof sonaar_music.option.show_artist_name != 'undefined' ? ' ' + sonaar_music.option.artist_separator + ' ' + list.tracks[currentTrack].track_artist:''}}{{list.tracks[currentTrack].album_title}}{{ currentTime }}{{ totalTime }}jQuery(function(jQuery){jQuery.datepicker.setDefaults({"closeText":"Close","currentText":"Today","monthNames":["January","February","March","April","May","June","July","August","September","October","November","December"],"monthNamesShort":["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"nextText":"Next","prevText":"Previous","dayNames":["Sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday"],"dayNamesShort":["Sun","Mon","Tue","Wed","Thu","Fri","Sat"],"dayNamesMin":["S","M","T","W","T","F","S"],"

In [31]:
def extract_all_data(url):    
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f'Error: Unable to fetch the page. Status code: {response.status_code}')
        return None

    html_content = response.content.decode('utf-8', 'ignore')
    tree = html.fromstring(html_content)
    headings = tree.xpath('//h1 | //h2 | //h3 |//h4 | //h5')

    result = {}

    for heading in headings:
        # Find all elements following the current heading until the next heading
        following_elements = heading.xpath('following-sibling::*[preceding-sibling::h1 or preceding-sibling::h2 or preceding-sibling::h3 or preceding-sibling::h4 or preceding-sibling::h5 or preceding-sibling::h6][1]')

        # Extract text from each following element
        extracted_text = []
        for el in following_elements:
            text = el.text_content().strip().replace("\t", '').replace("\n",'').replace("  ",'')
            if text:
                extracted_text.append(text)

        # Find all tables following the current heading until the next heading
        following_tables = heading.xpath('following-sibling::table')

        # Convert tables to strings
        extracted_tables = [lxml.etree.tostring(table).decode() for table in following_tables]

        # Store the result in the dictionary
        result[heading.text_content().replace("\t", '').replace("\n",'').replace("  ",'')] = {
            "text": extracted_text,
            "tables": extracted_tables
        }

    return result


In [35]:
res = extract_all_data(urls[2])
res

{'BROWSE BY TOPICS': {'text': ['Income Tax e-FilingLast Date To File ITR For 2022-23 Income Tax Slabs FY 2023-24Which Is Better: Old vs New Tax Regime For Salaried Employees?House PropertyBusiness, Professional &amp; FreelanceWhich ITR Should I File?Income Tax RefundsPaying Tax DueSalary IncomeCapital Gains IncomeOther income sources Advance TaxNRIHUFIncome Tax Notices'],
  'tables': []},
 'POPULAR ARTICLES': {'text': ['Budget 2023 Highlights: PDF Download, Key Takeaways, Important PointsBudget 2023: Full List of Cheaper and Costlier ItemsGST Collection October 2022GSTR 9C : Reconciliation Statement & Certification- Filing, Format & RulesGST Return – What is GST Return? Who Should File, Due Dates & Types of GST ReturnsGST Rates in 2022 – List of Goods and Service Tax Rates, Slab & RevisionEway Bill – What is Eway Bill? E Way Bill System, Rules & Generation Process ExplainedTypes of GST applicable on E-Commerce SalesGuide to File CMP-02 on GST Portalgst.gov.in : Government Website for G

In [51]:
url = 'https://www.transportpolicy.net/standard/india-fuels-diesel-and-gasoline/'
article = Article(url)
article.download()
article.parse()
article.nlp()
def extract_tables(html_content):
    tree = html.fromstring(article.html)

    tables = tree.xpath('//table')
    # print('tables', tables)
    table_data = []

    for table in tables:
        # Convert table to JSON format
        df = pd.read_html(html.tostring(table))[0]
        table_data.append(df.to_dict(orient='records'))

    return table_data if table_data else None

table = extract_tables(article.html)
print(table)

[[{'Date': 1995, 'Diesel': '10,000 ppm (nationwide)', 'Gasoline': '–'}, {'Date': 1996, 'Diesel': '5,000 ppm (Delhi + selected cities)', 'Gasoline': '–'}, {'Date': 1998, 'Diesel': '2,500 ppm (Delhi)', 'Gasoline': '–'}, {'Date': 1999, 'Diesel': '500 ppm (BS II, Delhi, limited supply)', 'Gasoline': '–'}, {'Date': 2000, 'Diesel': '2,500 ppm (nationwide)', 'Gasoline': '–'}, {'Date': 2001, 'Diesel': '500 ppm (BS II, selected cities)', 'Gasoline': '–'}, {'Date': 2005, 'Diesel': '500 ppm (BS II, nationwide) 350 ppm (BS III, selected cities)', 'Gasoline': '500 ppm (BS II, nationwide) 150 ppm (BS III, selected cities)'}, {'Date': 2010, 'Diesel': '350 ppm (BS III; nationwide) 50 ppm (BS IV; selected cities)', 'Gasoline': '150 ppm (BS III, nationwide) 50 ppm (BS IV, selected cities)'}, {'Date': 2017, 'Diesel': '50 ppm (BS IV;nationwide)', 'Gasoline': '50 ppm (BS IV; nationwide)'}, {'Date': 2020, 'Diesel': '10ppm (BS VI; nationwide)', 'Gasoline': '10 ppm (BS VI; nationwide)'}], [{'Characteristics':